In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import col


In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
productions_countries_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("countryId", StringType(), True)
])

In [0]:
productions_countries_df = spark.read \
    .schema(productions_countries_schema) \
    .option("multiline", "true") \
    .json(f"{bronze_folder_path}/{v_file_date}/production_country")

In [0]:
from pyspark.sql.functions import lit

In [0]:
productions_countries_fianl_df = add_ingestion_date(productions_countries_df) \
    .withColumnsRenamed({"countryId": "country_id", "movieId": "movie_id"}) \
    .withColumn("environment", lit(v_environment))\
    .withColumn("file_date", lit(v_file_date))

In [0]:
#overwrite_partition(productions_countries_fianl_df, "movie_silver", "production_country")

In [0]:
merge_condicion = "tgt.country_id = src.country_id AND tgt.movie_id = src.movie_id AND tgt.file_date = src.file_date"
merge_delta_lake(productions_countries_fianl_df, "movie_silver", "production_country", silver_folder_path, merge_condicion, "file_date")

In [0]:
%sql
SELECT file_date, COUNT(1)
FROM movie_silver.production_country
GROUP BY file_date;

In [0]:
%sql
SELECT * FROM movie_silver.production_country